# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - Membre 1
    - Membre 2
    - Membre 3




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [175]:
import numpy as np
import math
from math import ceil
import copy
from collections import deque
from random import randint
import random

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)
        
        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.score = 0
        
        self.rock = None

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.rock = self.rock
        return s

    def put_rock(self, rock_pos):
        s = copy.deepcopy(self)
        s.prev = self
        s.rock = rock_pos
        return s
    
    """
    estimee1: Estimation du nombre de coups restants 
    """
    def estimee1(self):
        
        return 4 - self.pos[0]
    
    
    """
    estimee2: estimee1 + nombre de voitures bloquantes
    """
    def estimee2(self, rh):
        
        return self.estimee1() + 5 * len(rh.blocking_cars(self))+4 * len(rh.blocking_cars_level_2(self)) + 3 *len(rh.blocking_cars_level_3(self))
    
    
    """
    estimee3: estimee1 + estimee2 + longueurs des voitures bloquantes
    """
    def estimee3(self, rh):
        
        block_len = 0
        for i in rh.blocking_cars(self):
            block_len += rh.length[i]
            
        return self.estimee1()+ self.estimee2(rh) + block_len
    
    
    """
    estimee4: estimee1 + estimee2 + estimee3 + coût maximal de la libération du chemin direct bloqué par les voitures
    """
    def estimee4(self, rh):
        move_away_cost = 0
        for i in rh.blocking_cars(self):
            move_away_cost_up = move_away_cost_down = 0
            if rh.move_on[0] - rh.length[i] >= 0:        #can we move it up?
                move_away_cost_up = self.pos[i]+rh.length[i]-rh.move_on[0]
            if rh.move_on[0] + rh.length[i] <= 5:        #can we move it down?
                move_away_cost_down = 1 - self.pos[i]+rh.move_on[0]
            if move_away_cost_up == 0 and move_away_cost_down > 0:
                move_away_cost += move_away_cost_down
            elif move_away_cost_up > 0 and move_away_cost_down == 0:
                move_away_cost += move_away_cost_up
            else:
                move_away_cost += min(move_away_cost_up,move_away_cost_down)
                    
            move_away_cost += min(move_away_cost_up,move_away_cost_down) #changed to min
        return move_away_cost
    
    def estimee5(self, rh):
        return self.estimee4(rh) + self.estimee1() + 5*len(rh.blocking_cars(self)) + 4*len(rh.blocking_cars_level_2(self)) + 3*len(rh.blocking_cars_level_3(self)) + 2*len(rh.blocking_cars_level_4(self)) + len(rh.blocking_cars_level_5(self))
     #+ self.estimee2(rh)+ self.estimee3(rh)
        
    def estimee6(self, rh):
        length = 0
        for set_var in rh.blocking_cars_level_general(self):
            length += len(set_var)
        return self.estimee1() + length + self.estimee4(rh)
    
    def estimee7(self, rh):
        L2,S2 = rh.blocking_cars_level_2(self)
        L3,S3= rh.blocking_cars_level_3(self)
        return self.estimee4(rh) +S2+S3+ self.estimee1()+5*len(rh.blocking_cars(self)) +4*len(L2) +3*len(L3) +2*len(rh.blocking_cars_level_4(self)) + 1*len(rh.blocking_cars_level_5(self))
    
    def move_blocker_away_cost(self,rh,blocked,blocker):
        move_away_cost = 0
        #for i in blocked.blockers:
        move_away_cost_up = move_away_cost_down = 0
        if rh.move_on[blocked] - rh.length[blocker] >= 0:        #can we move it up / left?
            move_away_cost_up = self.pos[blocker]+rh.length[blocker]-rh.move_on[blocked]
        if rh.move_on[blocked] + rh.length[blocker] <= 5:        #can we move it down /right?
            move_away_cost_down = 1 - self.pos[blocker]+rh.move_on[blocked]
        if move_away_cost_up == 0 and move_away_cost_down > 0:
            move_away_cost += move_away_cost_down
        elif move_away_cost_up > 0 and move_away_cost_down == 0:
            move_away_cost += move_away_cost_up
        else:
            move_away_cost += min(move_away_cost_up,move_away_cost_down)
        move_away_cost += min(move_away_cost_up,move_away_cost_down) #changed to min
        return move_away_cost
    
    def score_state(self, rh):
        self.score = 1000 + 10*self.pos[0] - self.estimee7(rh)
        if self.c == self.prev.c and self.d == -self.prev.d:
            self.score -= 50
        return self.score

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [176]:
class Blocker(object):
    def __init__(self, data):
        self.data = data
        self.blockers = set()

    def blocked_by(self, obj):
        return self.blockers.add(obj)

    def print_blockers(self, obj):
        listed = set()
        for i in self.blockers:
            listed.add(i.data)
        return listed
    
class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        if state.rock != None:
            self.free_pos[state.rock[0], state.rock[1]] = False
            
    def init_blocking_cars(self,state):
        blocked = 0
        blocker = Blocker(blocked)
        for i in range(self.nbcars):
            if not self.horiz[i]:
                if (state.pos[i]+self.length[i]-1 >= self.move_on[blocked]) and (state.pos[i] <= self.move_on[blocked]):
                    if self.move_on[i] > state.pos[blocked]+self.length[blocked]-1:
                        new_blocker = Blocker(i)
                        blocker.blocked_by(new_blocker)
        return blocker
    
    def blocking_cars(self,state):
        block_list = set()
        for i in range(self.nbcars):
            if not self.horiz[i]:
                if (state.pos[i]+self.length[i]-1 >= self.move_on[0]) and (state.pos[i] <= self.move_on[0]):
                    if self.move_on[i] > state.pos[0]+self.length[0]-1:
                        block_list.add(i)
        return block_list
    
    def blocking_cars_level_2(self,state):
        block_list = self.blocking_cars(state)
        block_list_level_2 = set()
        score = 0
        for i in range(1,self.nbcars):
            #if i in block_list: continue
            for blocker in block_list:
                if self.horiz[i]:
                    if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker]) and (state.pos[i] <= self.move_on[blocker]):
                        if ((self.move_on[i] > state.pos[blocker]+self.length[blocker]-1 ) and \
                            (self.move_on[i] <= self.move_on[0]+self.length[blocker])) or \
                        ((self.move_on[i] < state.pos[blocker]) and \
                         (self.move_on[i] >= self.move_on[0] - self.length[blocker])):
                            block_list_level_2.add(i)
                            score += state.move_blocker_away_cost(rh,blocker,i)
        return block_list_level_2,score
    
    def blocking_cars_level_3(self,state):
        score = 0
        block_list = self.blocking_cars(state)
        block_list_level_2 = set()
        block_list_level_2,a = self.blocking_cars_level_2(state)
        block_list_level_3 = set()
        for i in range(self.nbcars):
            #if i in block_list: continue
            for blocker in block_list:
                #if blocker in block_list_level_2: continue
                for blocker_level_2 in block_list_level_2:
                    if not self.horiz[i]:
                        if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_2]) and (state.pos[i] <= self.move_on[blocker_level_2]):
                            if ((self.move_on[i] > state.pos[blocker_level_2]+self.length[blocker_level_2]-1 ) and \
                                (self.move_on[i] <= self.move_on[blocker]+self.length[blocker_level_2])) or \
                            ((self.move_on[i] < state.pos[blocker_level_2]) and \
                             (self.move_on[i] >= self.move_on[blocker] - self.length[blocker_level_2])):
                                block_list_level_3.add(i)
                                score += state.move_blocker_away_cost(rh,blocker,i)
        return block_list_level_3,score
    
    def blocking_cars_level_4(self,state):
        #block_list = self.blocking_cars(state)
        block_list_level_2, _ = self.blocking_cars_level_2(state)
        block_list_level_3, _ = self.blocking_cars_level_3(state)
        block_list_level_4 = set()
        for i in range(1,self.nbcars):
            #if i in block_list: continue
            #for blocker in block_list:
                #if blocker in block_list_level_2: continue
            for blocker_level_2 in block_list_level_2:
                #if blocker_level_2 in block_list_level_3: continue
                for blocker_level_3 in block_list_level_3:
                    if self.horiz[i]:
                        if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_3]) and (state.pos[i] <= self.move_on[blocker_level_3]):
                            if ((self.move_on[i] > state.pos[blocker_level_3]+self.length[blocker_level_3]-1 ) and \
                                (self.move_on[i] <= self.move_on[blocker_level_2]+self.length[blocker_level_3])) or \
                            ((self.move_on[i] < state.pos[blocker_level_3]) and \
                             (self.move_on[i] >= self.move_on[blocker_level_2] - self.length[blocker_level_3])):
                                block_list_level_4.add(i)
        return block_list_level_4
    
    def blocking_cars_level_5(self,state):
        block_list_level_3,_ = self.blocking_cars_level_3(state)
        block_list_level_4 = self.blocking_cars_level_4(state)
        block_list_level_5 = set()
        for i in range(1,self.nbcars):
            #if i in block_list: continue
            #for blocker in block_list:
                #if blocker in block_list_level_2: continue
            for blocker_level_3 in block_list_level_3:
                #if blocker in block_list_level_3: continue
                for blocker_level_4 in block_list_level_4:
                    if not self.horiz[i]:
                        if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_4]) and (state.pos[i] <= self.move_on[blocker_level_4]):
                            if ((self.move_on[i] > state.pos[blocker_level_4]+self.length[blocker_level_4]-1 ) and \
                                (self.move_on[i] <= self.move_on[blocker_level_3]+self.length[blocker_level_4])) or \
                            ((self.move_on[i] < state.pos[blocker_level_4]) and \
                             (self.move_on[i] >= self.move_on[blocker_level_3] - self.length[blocker_level_4])):
                                block_list_level_5.add(i)
        return block_list_level_5
    
    def blocking_cars_level_general(self,state):
        blockers = []
        block_list = self.blocking_cars(state)
        block_list_level_2 = self.blocking_cars_level_2(state)
        vertical = True
        if block_list: blockers.append(block_list)
        if block_list_level_2: blockers.append(block_list_level_2)
        count = 2
        while block_list_level_2:
            block_list_level_3 = set()
            for i in range(self.nbcars):
                #if i in block_list: continue
                for blocker in block_list:
                    #if blocker in block_list_level_2: continue
                    for blocker_level_2 in block_list_level_2:
                        if ( (not self.horiz[i] and vertical) or ( self.horiz[i] and  not vertical)) :
                            if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_2]) and (state.pos[i] <= self.move_on[blocker_level_2]):
                                if ((self.move_on[i] > state.pos[blocker_level_2]+self.length[blocker_level_2]-1 ) and \
                                    (self.move_on[i] <= self.move_on[blocker]+self.length[blocker_level_2])) or \
                                ((self.move_on[i] < state.pos[blocker_level_2]) and \
                                 (self.move_on[i] >= self.move_on[blocker] - self.length[blocker_level_2])):
                                    block_list_level_3.add(i)
            #return block_list_level_3
            block_list = block_list_level_2
            block_list_level_2 = block_list_level_3
            vertical = not vertical
            if block_list_level_3 and len(blockers)<9:
                count+=1
                blockers.append(block_list_level_3)
                #print("appended new level :"+str(count))
            else: 
                #print("break from general")
                break
        return blockers
    
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states
    
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        for i in range(6):
            if i == 2: 
                continue
            for j in range(6):
                if self.free_pos[i,j] == True:
                    if state.rock == None:
                        new_states.append(state.put_rock((i,j)))
                    elif (i != state.rock[0]) and (j != state.rock[1]):
                        new_states.append(state.put_rock((i,j)))
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock != None:
            grid[state.rock] = 'x'
        print(grid)

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [177]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [178]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversariel avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [246]:
class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth

    def minimax_1(self, current_depth, current_state):
        #best_move = [current_state]
        if current_depth == 0 or current_state.success() == True:
            return current_state, current_state.score_state(self.rushhour)
        else:
            max_score = -100000
            possible_states = self.rushhour.possible_moves(current_state)
            #random.shuffle(possible_states)
            for p in possible_states:
                s, current_score = self.minimax_1(current_depth-1, p)
                
                '''temp = s
                while temp.prev != None:
                    if s == temp.prev:
                        current_score -= 50
                    temp.prev = temp.prev.prev'''
                    
                if current_score > max_score:
                    max_score = current_score
                    best_move = s
          
        #best_state = best_move[randint(0, len(best_move)-1)]
        self.print_move(True, best_move)
        return best_move, max_score
    
    def minimax_2(self, current_depth, current_state, is_max): 
        if current_depth == 0 or current_state.success() == True:
            return current_state, current_state.score_state(self.rushhour)
        if is_max:
            max_score = -100000
            possible_states = self.rushhour.possible_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                _, current_score = self.minimax_2(current_depth-1, p, False)
                
                temp = p
                while temp.prev != None:
                    if p == temp.prev:
                        current_score -= 1
                    temp.prev = temp.prev.prev
                    
                #if current_score == max_score:
                    #best_move_max.append(p)
                if current_score > max_score:
                    max_score = current_score
                    best_move_max = p
            
            #best_state = best_move_max[randint(0, len(best_move_max)-1)]
            #self.print_move(True, best_state)
            return best_move_max, max_score
        else:
            min_score = 100000
            possible_states = self.rushhour.possible_rock_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                _, current_score = self.minimax_2(current_depth-1, p, True)
                
                temp = p
                while temp.prev != None:
                    if p == temp.prev:
                        current_score -= 1
                    temp.prev = temp.prev.prev
                    
                #if current_score == min_score:
                    #best_move_min.append(p)
                if current_score < min_score:
                    min_score = current_score
                    best_move_min = p
            
            #best_state = best_move_min[randint(0, len(best_move_min)-1)]
            #self.print_move(False, best_state)
            return best_move_min, min_score

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        alpha = -100000
        beta = 100000
        if current_depth == 0 or current_state.success() == True:
            return current_state, current_state.score_state(self.rushhour)
        if is_max:
            max_score = -100000
            possible_states = self.rushhour.possible_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                _, current_score = self.minimax_pruning(current_depth-1, p, False, alpha, beta)
                
                temp = p
                while temp.prev != None:
                    if p == temp.prev:
                        current_score -= 1
                    temp.prev = temp.prev.prev
                    
                if current_score > max_score:
                    max_score = current_score
                    best_move_max = p
                    
                alpha = max(alpha, current_score)
                if beta < alpha:
                    break
                    
            return best_move_max, max_score
        else:
            min_score = 100000
            possible_states = self.rushhour.possible_rock_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                _, current_score = self.minimax_pruning(current_depth-1, p, True, alpha, beta)
                
                temp = p
                while temp.prev != None:
                    if p == temp.prev:
                        current_score -= 1
                    temp.prev = temp.prev.prev
                    
                if current_score < min_score:
                    min_score = current_score
                    best_move_min = p
                    
                beta = min(beta, current_score)
                if alpha < beta:
                    break
                    
            return best_move_min, min_score

    def expectimax(self, current_depth, current_state, is_max, fn_type):
        #fn_type: 0 -> random, 1 -> optimistic, 2 -> pessimistic
        if current_depth == 0 or current_state.success() == True:
            return current_state, current_state.score_state(self.rushhour)
        if is_max:
            #print("max agent of expectimax")
            max_score = -100000
            possible_states = self.rushhour.possible_moves(current_state)
            random.shuffle(possible_states)
            for p in possible_states:
                _, current_score = self.expectimax(current_depth-1, p, False, fn_type)
                
                temp = p
                while temp.prev != None:
                    if p == temp.prev:
                        current_score -= 1
                    temp.prev = temp.prev.prev
                    
                #if current_score == max_score:
                    #best_move_max.append(p)
                if current_score > max_score:
                    max_score = current_score
                    best_move_max = p
            return best_move_max, max_score
        else:
            if fn_type == "random": #random
                #print("random agent of expectimax")
                sum_scores = 0
                possible_states = self.rushhour.possible_rock_moves(current_state)
                random.shuffle(possible_states)
                probability = 1.0 / len(possible_states)
                for p in possible_states:
                    _, current_score = self.expectimax(current_depth-1, p, True, fn_type)

                    temp = p
                    while temp.prev != None:
                        if p == temp.prev:
                            current_score -= 1
                        temp.prev = temp.prev.prev

                    sum_scores += current_score * probability
                #current_state.score = sum_scores
                return current_state, sum_scores
            
            elif fn_type == "optimistic": #optimistic
                #print("optimistic agent of expectimax")
                max_score = -100000
                sum_scores = 0
                possible_states = self.rushhour.possible_rock_moves(current_state)
                random.shuffle(possible_states)
                #probability = ceil(len(possible_states)/2.0) / len(possible_states)
                probability = 0.75
                for p in possible_states:
                    _, current_score = self.expectimax(current_depth-1, p, True, fn_type)

                    temp = p
                    while temp.prev != None:
                        if p == temp.prev:
                            current_score -= 1
                        temp.prev = temp.prev.prev
                        
                    if current_score > max_score:
                        max_score = current_score
                        best_move_max = p
                
                sum_scores += best_move_max.score * probability
                for p in [x for x in possible_states if x != best_move_max]:
                    sum_scores +=  p.score * ((1 - probability)/(len(possible_states)-1))
                return current_state, sum_scores
            
            elif fn_type == "pessimistic": #pessimistic
                #print("pessimistic agent of expectimax")
                min_score = 100000
                sum_scores = 0
                possible_states = self.rushhour.possible_rock_moves(current_state)
                #probability = ceil(len(possible_states)/2.0) / len(possible_states)
                probability = 0.75
                for p in possible_states:
                    _, current_score = self.expectimax(current_depth-1, p, True, fn_type)

                    temp = p
                    while temp.prev != None:
                        if p == temp.prev:
                            current_score -= 1
                        temp.prev = temp.prev.prev
                        
                    if current_score < min_score:
                        min_score = current_score
                        best_move_min = p
                
                for b in best_move_min:
                    sum_scores += b.score * (probability/len(best_move_min))
                for p in [x for x in possible_states if x not in best_move_min]:
                    sum_scores +=  p.score * ((1 - probability)/len(possible_states - best_move_min))
                return current_state, sum_scores

    def decide_best_move_1(self, state):
        best_move, _ = self.minimax_1(self.search_depth, state)
        return best_move

    def decide_best_move_2(self, state, is_max):
        best_move, _ = self.minimax_2(self.search_depth, state, is_max)
        return best_move

    def decide_best_move_pruning(self, state, is_max, alpha, beta):
        best_move, _ = self.minimax_pruning(self.search_depth, state, is_max, alpha, beta)
        return best_move

    def decide_best_move_expectimax(self, state, is_max, fn_type):
        best_move, _ = self.expectimax(self.search_depth, state, is_max, fn_type)
        return best_move

    '''def solve(self, state, is_singleplayer, rh):
        best_moves = [state]
        next_best = state
        if is_singleplayer == True:
            while next_best.success() == False:
                print("solve here")
                next_best = self.decide_best_move_1(next_best, rh)
                print("solve dbm done")
                best_moves.append(next_best)
        print(len(best_moves))
        return best_moves'''
    
    def solve(self, state, is_singleplayer, rh):
        fifo = deque([state])
        count =0
        if is_singleplayer:
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                chosen_child = self.decide_best_move_1(p)
                #self.print_move(True,chosen_child)
                #rh.print_pretty_grid(chosen_child)
                count+=1
                fifo.append(chosen_child)
        else:
            flag = True
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                chosen_child = self.decide_best_move_2(p, flag)
                #self.print_move(True,chosen_child)
                #rh.print_pretty_grid(chosen_child)
                self.print_move(flag, chosen_child)
                if flag:
                    flag = False
                else:
                    flag = True
                count+=1
                fifo.append(chosen_child)
    
    def solve_pruning(self, state, is_singleplayer):
        fifo = deque([state])
        count =0
        alpha, beta = 0, 0
        if is_singleplayer:
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                chosen_child = self.decide_best_move_1(p)
                self.print_move(True,chosen_child)
                #rh.print_pretty_grid(chosen_child)
                count+=1
                fifo.append(chosen_child)
        else:
            alpha, beta = 0, 0
            flag = True
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                chosen_child = self.decide_best_move_pruning(p, flag, alpha, beta)
                #rh.print_pretty_grid(chosen_child)
                self.print_move(flag, chosen_child)
                if flag:
                    flag = False
                else:
                    flag = True
                count+=1
                fifo.append(chosen_child)
    
    def solve_expectimax(self, state, is_singleplayer, fn_type):
        fifo = deque([state])
        count =0
        if is_singleplayer:
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                chosen_child = self.decide_best_move_1(p)
                self.print_move(True,chosen_child)
                #rh.print_pretty_grid(chosen_child)
                count+=1
                fifo.append(chosen_child)
        else:
            flag = True
            while fifo:
                p=fifo.popleft()
                if p.success():
                    print("Nombre de mouvements: "+str(count))
                    return p
                chosen_child = self.decide_best_move_expectimax(p, flag, fn_type)
                #rh.print_pretty_grid(chosen_child)
                self.print_move(flag, chosen_child)
                if flag:
                    flag = False
                else:
                    flag = True
                count+=1
                fifo.append(chosen_child)
    
    def print_move(self, is_max, state):
        if state:
            if is_max:
                msg = "Voiture "
                msg += self.rushhour.color[state.c] + " vers "
                if self.rushhour.horiz[state.c]:
                    if state.d == 1:
                        msg += "la droite"
                    else:
                        msg += "la gauche"
                else:
                    if state.d ==1:
                        msg += "le bas"
                    else:
                        msg += "le haut"
                print(msg)
            elif state.rock:
                msg = "Roche dans la case "+str(state.rock[0])+"-"+str(state.rock[1])
                print(msg)
        return None


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [116]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

Voiture rouge vers la droite
Roche dans la case 3-1


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [250]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True, rh)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
Voiture rouge vers la droite
Voiture bleu vers le haut
Voiture orange vers le haut
Voiture jaune vers la gauche
Voiture vert vers le bas
Voiture vert vers le bas
Voiture vert vers le bas
Voiture rouge vers la droite
Voiture rouge vers la droite
Nombre de mouvements: 9
Wall time: 0 ns


In [252]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True, rh) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
Voiture emeraude vers la gauche
Voiture jaune vers le bas
Voiture vert vers la droite
Voiture mauve vers le haut
Voiture orange vers le haut
Voiture emeraude vers la gauche
Voiture lime vers la gauche
Voiture lime vers la gauche
Voiture lime vers la gauche
Voiture jaune vers le bas
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture bleu vers le bas
Voiture rouge vers la droite
Voiture rouge vers la droite
Voiture rouge vers la droite
Nombre de mouvements: 16
Wall time: 0 ns


In [253]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True, rh)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
Voiture rouge vers la droite
Voiture 1 vers le bas
Voiture 2 vers la gauche
Voiture 2 vers la gauche
Voiture 2 vers la gauche
Voiture 3 vers le haut
Voiture rouge vers la droite
Voiture 10 vers la gauche
Voiture 4 vers le haut
Voiture 4 vers le haut
Voiture rouge vers la droite
Voiture 5 vers le bas
Voiture 5 vers le bas
Voiture rouge vers la droite
Nombre de mouvements: 14
Wall time: 0 ns


### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [226]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, rh)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
Voiture bleu vers le haut
Roche dans la case 4-4
Voiture rouge vers la droite
Roche dans la case 1-0
Voiture vert vers le bas
Roche dans la case 0-2
Voiture vert vers le bas
Roche dans la case 5-0
Voiture orange vers le haut
Roche dans la case 0-4
Voiture jaune vers la gauche
Roche dans la case 1-2
Voiture vert vers le bas
Roche dans la case 0-4
Voiture rouge vers la droite
Roche dans la case 5-0
Voiture rouge vers la droite
Nombre de mouvements: 17
Wall time: 0 ns


In [579]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, rh) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
Voiture jaune vers le bas
Roche dans la case 5-1
Voiture vert vers la droite
Roche dans la case 4-3
Voiture mauve vers le haut
Roche dans la case 3-0
Voiture emeraude vers la gauche
Roche dans la case 0-3
Voiture orange vers le haut
Roche dans la case 5-0
Voiture lime vers la gauche
Roche dans la case 3-4
Voiture emeraude vers la gauche
Roche dans la case 1-2
Voiture jaune vers le bas
Roche dans la case 5-5
Voiture lime vers la gauche
Roche dans la case 0-3
Voiture lime vers la gauche
Roche dans la case 5-5
Voiture bleu vers le bas
Roche dans la case 0-4
Voiture bleu vers le bas
Roche dans la case 5-5
Voiture rouge vers la droite
Roche dans la case 1-2
Voiture jaune vers le bas
Roche dans la case 5-4
Voiture rouge vers la droite
Roche dans la case 1-2
Voi

In [572]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, rh)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
Voiture rouge vers la droite
Roche dans la case 1-4
Voiture 1 vers le bas
Roche dans la case 4-0
Voiture 4 vers le haut
Roche dans la case 3-3
Voiture 2 vers la gauche
Roche dans la case 1-2
Voiture 2 vers la gauche
Roche dans la case 5-5
Voiture 2 vers la gauche
Roche dans la case 1-2
Voiture 3 vers le haut
Roche dans la case 0-4
Voiture 10 vers la gauche
Roche dans la case 4-5
Voiture 4 vers le haut
Roche dans la case 1-2
Voiture 5 vers le bas
Roche dans la case 5-5
Voiture rouge vers la droite
Roche dans la case 4-1
Voiture 5 vers le bas
Roche dans la case 3-3
Voiture rouge vers la droite
Roche dans la case 1-2
Voiture rouge vers la droite
Nombre de mouvements: 27
Wall time: 0 ns


## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [248]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_pruning(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
Voiture bleu vers le haut
Roche dans la case 1-3
Voiture rouge vers la droite
Roche dans la case 0-2
Voiture orange vers le haut
Roche dans la case 3-5
Voiture jaune vers la gauche
Roche dans la case 0-3
Voiture vert vers le bas
Roche dans la case 1-2
Voiture vert vers le bas
Roche dans la case 5-5
Voiture vert vers le bas
Roche dans la case 0-2
Voiture rouge vers la droite
Roche dans la case 1-3
Voiture rouge vers la droite
Nombre de mouvements: 17
Wall time: 0 ns


In [249]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_pruning(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
Voiture emeraude vers la gauche
Roche dans la case 0-2
Voiture jaune vers le bas
Roche dans la case 1-1
Voiture bleu vers le haut
Roche dans la case 5-5
Voiture bleu vers le bas
Roche dans la case 1-1
Voiture vert vers la droite
Roche dans la case 4-2
Voiture mauve vers le haut
Roche dans la case 3-1
Voiture orange vers le haut
Roche dans la case 5-4
Voiture lime vers la gauche
Roche dans la case 4-2
Voiture lime vers la droite
Roche dans la case 1-1
Voiture bleu vers le haut
Roche dans la case 5-0
Voiture bleu vers le bas
Roche dans la case 4-3
Voiture emeraude vers la gauche
Roche dans la case 0-4
Voiture bleu vers le bas
Roche dans la case 1-1
Voiture bleu vers le bas
Roche dans la case 3-2
Voiture rouge vers la droite
Roche dans la case 1-3
Voiture ve

Voiture vert vers la droite
Roche dans la case 1-4
Voiture vert vers la droite
Roche dans la case 3-1
Voiture vert vers la gauche
Roche dans la case 0-5
Voiture vert vers la droite
Roche dans la case 4-2
Voiture vert vers la gauche
Roche dans la case 3-4
Voiture vert vers la droite
Roche dans la case 1-3
Voiture vert vers la gauche
Roche dans la case 3-2
Voiture jaune vers le haut
Roche dans la case 1-1
Voiture jaune vers le bas
Roche dans la case 4-2
Voiture jaune vers le haut
Roche dans la case 5-5
Voiture vert vers la droite
Roche dans la case 1-4
Voiture vert vers la gauche
Roche dans la case 3-5
Voiture vert vers la droite
Roche dans la case 1-4
Voiture jaune vers le bas
Roche dans la case 5-5
Voiture vert vers la gauche
Roche dans la case 3-4
Voiture jaune vers le haut
Roche dans la case 4-1
Voiture jaune vers le bas
Roche dans la case 3-4
Voiture vert vers la gauche
Roche dans la case 1-1
Voiture vert vers la droite
Roche dans la case 0-5
Voiture vert vers la gauche
Roche dans l

Roche dans la case 3-1
Voiture vert vers la gauche
Roche dans la case 5-5
Voiture vert vers la droite
Roche dans la case 1-1
Voiture jaune vers le haut
Roche dans la case 5-5
Voiture vert vers la gauche
Roche dans la case 3-1
Voiture vert vers la droite
Roche dans la case 5-4
Voiture vert vers la gauche
Roche dans la case 4-2
Voiture vert vers la droite
Roche dans la case 3-1
Voiture vert vers la droite
Roche dans la case 1-2
Voiture vert vers la gauche
Roche dans la case 4-1
Voiture vert vers la gauche
Roche dans la case 3-2
Voiture vert vers la droite
Roche dans la case 1-3
Voiture vert vers la gauche
Roche dans la case 3-1
Voiture jaune vers le bas
Roche dans la case 1-2
Voiture vert vers la droite
Roche dans la case 5-5
Voiture vert vers la droite
Roche dans la case 3-4
Voiture vert vers la gauche
Roche dans la case 1-2
Voiture vert vers la droite
Roche dans la case 0-5
Voiture vert vers la gauche
Roche dans la case 5-4
Voiture jaune vers le haut
Roche dans la case 1-2
Voiture vert

Roche dans la case 3-4
Voiture vert vers la droite
Roche dans la case 4-2
Voiture vert vers la gauche
Roche dans la case 1-3
Voiture jaune vers le bas
Roche dans la case 4-1
Voiture vert vers la droite
Roche dans la case 3-4
Voiture vert vers la gauche
Roche dans la case 1-2
Voiture jaune vers le haut
Roche dans la case 4-1
Voiture vert vers la droite
Roche dans la case 5-5
Voiture vert vers la gauche
Roche dans la case 4-1
Voiture vert vers la gauche
Roche dans la case 1-2
Voiture jaune vers le bas
Roche dans la case 0-5
Voiture vert vers la droite
Roche dans la case 5-4
Voiture jaune vers le haut
Roche dans la case 3-2
Voiture jaune vers le bas
Roche dans la case 1-3
Voiture vert vers la droite
Roche dans la case 3-2
Voiture vert vers la gauche
Roche dans la case 5-5
Voiture vert vers la gauche
Roche dans la case 3-4
Voiture vert vers la droite
Roche dans la case 1-2
Voiture jaune vers le haut
Roche dans la case 3-5
Voiture vert vers la gauche
Roche dans la case 0-4
Voiture vert vers

Voiture vert vers la droite
Roche dans la case 4-2
Voiture jaune vers le haut
Roche dans la case 0-1
Voiture jaune vers le bas
Roche dans la case 1-2
Voiture vert vers la droite
Roche dans la case 0-5
Voiture vert vers la gauche
Roche dans la case 1-1
Voiture jaune vers le haut
Roche dans la case 5-4
Voiture vert vers la gauche
Roche dans la case 3-1
Voiture vert vers la droite
Roche dans la case 4-2
Voiture vert vers la gauche
Roche dans la case 5-5
Voiture vert vers la droite
Roche dans la case 3-1
Voiture jaune vers le bas
Roche dans la case 1-2
Voiture jaune vers le haut
Roche dans la case 3-4
Voiture vert vers la gauche
Roche dans la case 5-5
Voiture vert vers la droite
Roche dans la case 3-1
Voiture vert vers la droite
Roche dans la case 1-2
Voiture vert vers la gauche
Roche dans la case 4-1
Voiture vert vers la gauche
Roche dans la case 3-5
Voiture vert vers la droite
Roche dans la case 0-4
Voiture jaune vers le bas
Roche dans la case 5-5
Voiture jaune vers le haut
Roche dans la

...

Voiture jaune vers le bas
Roche dans la case 0-5
Voiture vert vers la gauche
Roche dans la case 1-4
Voiture vert vers la gauche
Roche dans la case 5-5
Voiture vert vers la droite
Roche dans la case 1-2
Voiture vert vers la gauche
Roche dans la case 4-1
Voiture vert vers la droite
Roche dans la case 1-3
Voiture jaune vers le haut
Roche dans la case 3-4
Voiture vert vers la droite
Roche dans la case 0-1
Voiture vert vers la gauche
Roche dans la case 1-4
Voiture vert vers la gauche
Roche dans la case 4-1
Voiture vert vers la droite
Roche dans la case 1-4
Voiture vert vers la droite
Roche dans la case 0-1
Voiture jaune vers le bas
Roche dans la case 1-4
Voiture vert vers la gauche
Roche dans la case 0-5
Voiture vert vers la droite
Roche dans la case 1-1
Voiture vert vers la gauche
Roche dans la case 0-4
Voiture jaune vers le haut
Roche dans la case 3-5
Voiture vert vers la gauche
Roche dans la case 0-4
Voiture vert vers la droite
Roche dans la case 1-1
Voiture vert vers la gauche
Roche dan

Roche dans la case 4-1
Voiture jaune vers le bas
Roche dans la case 0-2
Voiture jaune vers le haut
Roche dans la case 5-4
Voiture jaune vers le bas
Roche dans la case 1-1
Voiture vert vers la gauche
Roche dans la case 0-5
Voiture vert vers la droite
Roche dans la case 3-4
Voiture jaune vers le haut
Roche dans la case 4-2
Voiture jaune vers le bas
Roche dans la case 1-4
Voiture vert vers la gauche
Roche dans la case 4-2
Voiture jaune vers le haut
Roche dans la case 3-4
Voiture vert vers la droite
Roche dans la case 4-1
Voiture vert vers la gauche
Roche dans la case 3-2
Voiture vert vers la gauche
Roche dans la case 5-4
Voiture jaune vers le bas
Roche dans la case 3-1
Voiture vert vers la droite
Roche dans la case 1-3
Voiture vert vers la droite
Roche dans la case 0-1
Voiture vert vers la gauche
Roche dans la case 5-4
Voiture vert vers la droite
Roche dans la case 4-2
Voiture vert vers la gauche
Roche dans la case 1-4
Voiture jaune vers le haut
Roche dans la case 3-5
Voiture vert vers la

Voiture jaune vers le haut
Roche dans la case 5-4
Voiture jaune vers le bas
Roche dans la case 1-1
Voiture vert vers la gauche
Roche dans la case 5-5
Voiture vert vers la droite
Roche dans la case 3-2
Voiture vert vers la gauche
Roche dans la case 1-1
Voiture vert vers la droite
Roche dans la case 3-4
Voiture jaune vers le haut
Roche dans la case 1-1
Voiture jaune vers le bas
Roche dans la case 3-2
Voiture vert vers la droite
Roche dans la case 1-4
Voiture vert vers la gauche
Roche dans la case 3-2
Voiture jaune vers le haut
Roche dans la case 4-1
Voiture vert vers la gauche
Roche dans la case 5-4
Voiture jaune vers le bas
Roche dans la case 1-1
Voiture vert vers la droite
Roche dans la case 5-4
Voiture vert vers la gauche
Roche dans la case 0-5
Voiture vert vers la droite
Roche dans la case 1-4
Voiture vert vers la gauche
Roche dans la case 4-2
Voiture vert vers la droite
Roche dans la case 3-4
Voiture jaune vers le haut
Roche dans la case 0-1
Voiture jaune vers le bas
Roche dans la c

Voiture vert vers la gauche
Roche dans la case 4-2
Voiture vert vers la droite
Roche dans la case 0-5
Voiture vert vers la gauche
Roche dans la case 3-2
Voiture vert vers la droite
Roche dans la case 1-1
Voiture vert vers la gauche
Roche dans la case 0-5
Voiture vert vers la droite
Roche dans la case 5-4
Voiture vert vers la gauche
Roche dans la case 1-1
Voiture jaune vers le haut
Roche dans la case 3-2
Voiture vert vers la droite
Roche dans la case 1-1
Voiture vert vers la gauche
Roche dans la case 5-4
Voiture jaune vers le bas
Roche dans la case 1-3
Voiture vert vers la gauche
Roche dans la case 3-2
Voiture vert vers la droite
Roche dans la case 0-1
Voiture jaune vers le haut
Roche dans la case 3-2
Voiture vert vers la gauche
Roche dans la case 5-4
Voiture vert vers la droite
Roche dans la case 3-1
Voiture jaune vers le bas
Roche dans la case 0-4
Voiture jaune vers le haut
Roche dans la case 1-1
Voiture vert vers la droite
Roche dans la case 3-4
Voiture jaune vers le bas
Roche dans l

KeyboardInterrupt: 

In [237]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_pruning(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
Voiture rouge vers la droite
Roche dans la case 4-0
Voiture 3 vers le bas
Roche dans la case 0-1
Voiture 3 vers le bas
Roche dans la case 5-5
Voiture rouge vers la droite
Roche dans la case 1-2
Voiture 2 vers la gauche
Roche dans la case 0-5
Voiture 4 vers le haut
Roche dans la case 4-1
Voiture 2 vers la gauche
Roche dans la case 3-4
Voiture 4 vers le haut
Roche dans la case 4-0
Voiture 1 vers le bas
Roche dans la case 1-3
Voiture 1 vers le haut
Roche dans la case 4-1
Voiture rouge vers la droite
Roche dans la case 3-4
Voiture 1 vers le bas
Roche dans la case 4-1
Voiture 2 vers la gauche
Roche dans la case 0-3
Voiture 5 vers le haut
Roche dans la case 1-1
Voiture 5 vers le bas
Roche dans la case 4-0
Voiture 2 vers la droite
Roche dans la case 1-1
Voiture 

Roche dans la case 4-0
Voiture 5 vers le haut
Roche dans la case 3-5
Voiture 1 vers le haut
Roche dans la case 4-0
Voiture 1 vers le bas
Roche dans la case 1-3
Voiture 2 vers la gauche
Roche dans la case 3-4
Voiture 2 vers la droite
Roche dans la case 1-2
Voiture 1 vers le haut
Roche dans la case 3-4
Voiture 5 vers le bas
Roche dans la case 4-0
Voiture 1 vers le bas
Roche dans la case 1-1
Voiture 2 vers la gauche
Roche dans la case 0-5
Voiture 2 vers la droite
Roche dans la case 4-0
Voiture 5 vers le haut
Roche dans la case 3-5
Voiture 1 vers le haut
Roche dans la case 4-0
Voiture 1 vers le bas
Roche dans la case 5-5
Voiture 5 vers le bas
Roche dans la case 1-2
Voiture 5 vers le haut
Roche dans la case 3-4
Voiture 1 vers le haut
Roche dans la case 4-1
Voiture 1 vers le bas
Roche dans la case 3-5
Voiture 1 vers le haut
Roche dans la case 4-0
Voiture 1 vers le bas
Roche dans la case 1-1
Voiture 5 vers le bas
Roche dans la case 0-5
Voiture 2 vers la gauche
Roche dans la case 1-1
Voiture 2

Voiture 5 vers le bas
Roche dans la case 1-3
Voiture 1 vers le haut
Roche dans la case 5-5
Voiture 1 vers le bas
Roche dans la case 4-1
Voiture 2 vers la gauche
Roche dans la case 3-4
Voiture 2 vers la droite
Roche dans la case 5-5
Voiture 1 vers le haut
Roche dans la case 3-4
Voiture 1 vers le bas
Roche dans la case 4-0
Voiture 1 vers le haut
Roche dans la case 5-5
Voiture 1 vers le bas
Roche dans la case 0-0
Voiture 5 vers le haut
Roche dans la case 3-5
Voiture 2 vers la gauche
Roche dans la case 4-1
Voiture 5 vers le bas
Roche dans la case 0-5
Voiture 2 vers la droite
Roche dans la case 3-4
Voiture 1 vers le haut
Roche dans la case 4-1
Voiture 1 vers le bas
Roche dans la case 0-0
Voiture 5 vers le haut
Roche dans la case 3-4
Voiture 5 vers le bas
Roche dans la case 1-1
Voiture 5 vers le haut
Roche dans la case 3-4
Voiture 5 vers le bas
Roche dans la case 1-1
Voiture 5 vers le haut
Roche dans la case 5-5
Voiture 2 vers la gauche
Roche dans la case 0-3
Voiture 5 vers le bas
Roche dans

Roche dans la case 0-0
Voiture 5 vers le haut
Roche dans la case 5-5
Voiture 2 vers la gauche
Roche dans la case 0-3
Voiture 5 vers le bas
Roche dans la case 1-2
Voiture 2 vers la droite
Roche dans la case 0-5
Voiture 1 vers le haut
Roche dans la case 4-0
Voiture 1 vers le bas
Roche dans la case 3-4
Voiture 2 vers la gauche
Roche dans la case 0-3
Voiture 5 vers le haut
Roche dans la case 3-5
Voiture 2 vers la droite
Roche dans la case 4-0
Voiture 2 vers la gauche
Roche dans la case 1-1
Voiture 5 vers le bas
Roche dans la case 0-3
Voiture 5 vers le haut
Roche dans la case 4-0
Voiture 5 vers le bas
Roche dans la case 1-1
Voiture 2 vers la droite
Roche dans la case 5-5
Voiture 1 vers le haut
Roche dans la case 4-1
Voiture 1 vers le bas
Roche dans la case 0-5
Voiture 1 vers le haut
Roche dans la case 4-0
Voiture 1 vers le bas
Roche dans la case 5-5
Voiture 5 vers le haut
Roche dans la case 1-3
Voiture 2 vers la gauche
Roche dans la case 4-1
Voiture 5 vers le bas
Roche dans la case 1-2
Voit

Voiture 5 vers le bas
Roche dans la case 5-5
Voiture 1 vers le bas
Roche dans la case 4-0
Voiture 1 vers le haut
Roche dans la case 0-5
Voiture 1 vers le bas
Roche dans la case 4-1
Voiture 5 vers le haut
Roche dans la case 5-5
Voiture 1 vers le haut
Roche dans la case 1-2
Voiture 5 vers le bas
Roche dans la case 4-1
Voiture 1 vers le bas
Roche dans la case 3-4
Voiture 2 vers la gauche
Roche dans la case 1-1
Voiture 2 vers la droite
Roche dans la case 0-0
Voiture 5 vers le haut
Roche dans la case 3-4
Voiture 5 vers le bas
Roche dans la case 1-1
Voiture 1 vers le haut
Roche dans la case 5-5
Voiture 1 vers le bas
Roche dans la case 0-0
Voiture 5 vers le haut
Roche dans la case 5-5
Voiture 1 vers le haut
Roche dans la case 3-4
Voiture 5 vers le bas
Roche dans la case 1-3
Voiture 1 vers le bas
Roche dans la case 4-0
Voiture 1 vers le haut
Roche dans la case 0-5
Voiture 1 vers le bas
Roche dans la case 4-1
Voiture 5 vers le haut
Roche dans la case 5-5
Voiture 2 vers la gauche
Roche dans la c

Roche dans la case 5-5
Voiture 1 vers le haut
Roche dans la case 1-3
Voiture 1 vers le bas
Roche dans la case 3-4
Voiture 1 vers le haut
Roche dans la case 1-1
Voiture 1 vers le bas
Roche dans la case 5-5
Voiture 5 vers le haut
Roche dans la case 1-1
Voiture 2 vers la gauche
Roche dans la case 3-4
Voiture 5 vers le bas
Roche dans la case 1-2
Voiture 2 vers la droite
Roche dans la case 4-0
Voiture 5 vers le haut
Roche dans la case 3-5
Voiture 2 vers la gauche
Roche dans la case 1-3
Voiture 5 vers le bas
Roche dans la case 4-1
Voiture 2 vers la droite
Roche dans la case 1-3
Voiture 2 vers la gauche
Roche dans la case 4-0
Voiture 2 vers la droite
Roche dans la case 3-4
Voiture 1 vers le haut
Roche dans la case 4-0
Voiture 1 vers le bas
Roche dans la case 1-3
Voiture 1 vers le haut
Roche dans la case 0-5
Voiture 1 vers le bas
Roche dans la case 4-0
Voiture 5 vers le haut
Roche dans la case 1-3
Voiture 2 vers la gauche
Roche dans la case 5-5
Voiture 2 vers la droite
Roche dans la case 0-0
V

KeyboardInterrupt: 

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [254]:
# solution optimale: 9 moves

rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_expectimax(s, False, "optimistic")
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
Voiture rouge vers la droite
Voiture bleu vers le haut
Voiture vert vers le bas
Voiture vert vers le haut
Voiture rouge vers la gauche
Voiture rouge vers la droite
Voiture jaune vers la droite
Voiture vert vers le bas
Voiture jaune vers la gauche
Voiture vert vers le bas
Voiture vert vers le haut
Voiture rouge vers la gauche
Voiture bleu vers le bas
Voiture vert vers le haut
Voiture vert vers le bas
Voiture rouge vers la gauche
Voiture vert vers le haut
Voiture rouge vers la droite
Voiture bleu vers le bas
Voiture vert vers le bas
Voiture bleu vers le bas
Voiture rouge vers la gauche


KeyboardInterrupt: 

In [255]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_expectimax(s, False, "optimistic") 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
Voiture bleu vers le haut
Voiture vert vers la droite
Voiture lime vers la gauche
Voiture bleu vers le bas


KeyboardInterrupt: 

In [256]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve_expectimax(s, False, "optimistic")
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
Voiture rouge vers la droite
Voiture 6 vers la droite
Voiture 6 vers la gauche
Voiture 9 vers la droite
Voiture 9 vers la gauche
Voiture 1 vers le bas
Voiture 10 vers la gauche
Voiture 6 vers la droite
Voiture 1 vers le haut
Voiture 9 vers la droite
Voiture 2 vers la gauche
Voiture 10 vers la droite
Voiture 1 vers le bas
Voiture 6 vers la gauche
Voiture 10 vers la gauche
Voiture 10 vers la droite
Voiture 10 vers la gauche
Voiture 5 vers le bas
Voiture 6 vers la droite
Voiture 5 vers le haut
Voiture 1 vers le bas
Voiture 1 vers le bas
Voiture 2 vers la droite
Voiture 9 vers la gauche
Voiture 10 vers la droite
Voiture 1 vers le haut
Voiture 9 vers la droite
Voiture 4 vers le haut
Voiture 1 vers le bas
Voiture 1 vers le haut
Voiture 10 vers la gauche
Voiture

Voiture 2 vers la gauche
Voiture 10 vers la droite
Voiture 4 vers le haut
Voiture 1 vers le bas
Voiture 2 vers la gauche
Voiture 2 vers la droite
Voiture 7 vers la droite
Voiture 7 vers la gauche
Voiture 4 vers le bas
Voiture 7 vers la droite
Voiture 1 vers le haut
Voiture 1 vers le bas
Voiture 2 vers la gauche
Voiture rouge vers la gauche
Voiture 3 vers le haut
Voiture 3 vers le bas
Voiture 1 vers le bas
Voiture 8 vers le haut
Voiture 3 vers le haut
Voiture 8 vers le haut
Voiture 10 vers la gauche
Voiture 7 vers la gauche
Voiture 1 vers le haut
Voiture 10 vers la droite
Voiture 10 vers la gauche
Voiture 9 vers la gauche
Voiture 9 vers la droite
Voiture 10 vers la gauche
Voiture 10 vers la gauche
Voiture 7 vers la gauche
Voiture 10 vers la droite
Voiture 4 vers le haut
Voiture 9 vers la gauche
Voiture 10 vers la droite
Voiture 9 vers la gauche
Voiture 8 vers le bas
Voiture 8 vers le haut
Voiture 3 vers le bas
Voiture 8 vers le bas
Voiture 8 vers le bas
Voiture 4 vers le bas
Voiture 10 

Voiture 5 vers le haut
Voiture 1 vers le haut
Voiture 9 vers la gauche
Voiture 2 vers la droite
Voiture 5 vers le bas
Voiture 2 vers la gauche
Voiture 9 vers la droite
Voiture 10 vers la droite
Voiture 7 vers la droite
Voiture 2 vers la droite
Voiture 10 vers la gauche
Voiture 2 vers la gauche
Voiture 1 vers le bas
Voiture 2 vers la droite
Voiture 2 vers la droite
Voiture 2 vers la gauche
Voiture 5 vers le haut
Voiture 6 vers la droite
Voiture 1 vers le haut
Voiture 2 vers la gauche
Voiture 2 vers la gauche
Voiture 3 vers le haut
Voiture 1 vers le bas
Voiture 8 vers le bas
Voiture 4 vers le haut
Voiture 1 vers le haut


KeyboardInterrupt: 

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 